In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_columns',None)

In [3]:
#读取数据
data_floder = "F:\\downloads\\ChromeDownload\\huaweicontest\\data\\"

age_train = pd.read_csv(data_floder +"age_train.csv", names=['uid','age_group'])
age_test = pd.read_csv(data_floder + "age_test.csv", names=['uid'])
user_basic_info = pd.read_csv(data_floder + "user_basic_info.csv", names=['uid','gender','city','prodName','ramCapacity','ramLeftRation','romCapacity','romLeftRation','color','fontSize','ct','carrier','os'])
user_behavior_info = pd.read_csv(data_floder + "user_behavior_info.csv", names=['uid','bootTimes','AFuncTimes','BFuncTimes','CFuncTimes','DFuncTimes','EFuncTimes','FFuncTimes','FFuncSum'])
user_app_actived = pd.read_csv(data_floder + "user_app_actived.csv", names=['uid','appId'])
#user_app_usage = pd.read_csv("user_app_usage.csv")
app_info = pd.read_csv(data_floder + "app_info.csv", names=['appId', 'category'])

# 数据清洗

ramCapacity = user_basic_info['ramCapacity'].values
for i in range(len(ramCapacity)):
    if ramCapacity[i] == 0.0:
        ramCapacity[i] = 1.0
user_basic_info['ramCapacity'] = ramCapacity

ramLeftRation = user_basic_info['ramLeftRation'].values
for i in range(len(ramLeftRation)):
    if ramLeftRation[i] > 1.0:
        ramLeftRation[i] = 1.0
user_basic_info['ramLeftRation'] = ramLeftRation
user_basic_info['ramLeftRation'] = user_basic_info['ramLeftRation'].fillna(user_basic_info['ramLeftRation'].mean())

romLeftRation = user_basic_info['romLeftRation'].values
for i in range(len(romLeftRation)):
    if romLeftRation[i] > 1.0:
        romLeftRation[i] = 1.0
user_basic_info['romLeftRation'] = romLeftRation
user_basic_info['romLeftRation'] = user_basic_info['romLeftRation'].fillna(user_basic_info['romLeftRation'].mean())

user_basic_info['fontSize'] = user_basic_info['fontSize'].fillna(1.45)

user_basic_info = user_basic_info.fillna(method='ffill')


In [5]:
from sklearn import preprocessing

def processUserAppActived():
    le = preprocessing.LabelEncoder()
    le.fit(app_info['category'].values.tolist())

    # 可以查看一下 fit 以后的类别是什么
    # le.classes_

    # transform 以后，这一列数就变成了 [0,  n-1] 这个区间的数，即是  le.classes_ 中的索引
    app_info['category'] = le.transform(app_info['category'].values.tolist())

    # 转化 app_info，便于之后根据 appId 索引到对应的类别（多个）
    app_dict = app_info.groupby('appId')['category'].apply(list)
    
    # 新建 0 数组，记录每位用户对 40 个类别的应用的安装数量
    app_count = np.zeros([2512500, 40], dtype=np.int8)

    # 所有用户的表示安装app的标识
    appId = user_app_actived['appId'].values

    for i in range(len(appId)): # 每个用户依次处理
        # 分割 appId 串
        app_ids = np.array(appId[i].split('#'))
        for app_id in app_ids:
            if app_id in app_dict:
                # 获取该 app_id 对应的类别 list
                categories = app_dict[app_id]
                for category in categories:
                    # 该用户该类别应用数量加一
                    app_count[i][category] += 1
                    
    # 保存数据
    df1 = user_app_actived.drop('appId', axis=1)
    df2 = pd.DataFrame(app_count)
    df = pd.concat([df1, df2], axis=1)
    df.to_csv('appActivedProcessed.csv',index=False)

In [8]:
#处理数据量较大的user_app_usage.csv，结合app_info.csv简单统计得到appuseProcessed.csv作为特征
def f(x):
    s = x.value_counts()
    return np.nan if len(s) == 0 else s.index[0]
def processUserAppUsage():
    resTable = pd.DataFrame()
    reader = pd.read_csv(data_floder + "user_app_usage.csv", names=['uid','appId','duration','times','use_date'], iterator=True)
    last_df = pd.DataFrame()
    
    app_info = pd.read_csv(data_floder + "app_info.csv", names=['appId','category'])
    cats = list(set(app_info['category']))
    category2id = dict(zip(sorted(cats), range(0,len(cats))))
    id2category = dict(zip(range(0,len(cats)), sorted(cats)))
    app_info['category'] = app_info['category'].apply(lambda x: category2id[x])
    i = 1
    
    while True:
        try:
            print("index: {}".format(i))
            i+=1
            df = reader.get_chunk(1000000)
            df = pd.concat([last_df, df])
            idx = df.shape[0]-1
            last_user = df.iat[idx,0]
            while(df.iat[idx,0]==last_user):
                idx-=1
            last_df = df[idx+1:]
            df = df[:idx+1]

            now_df = pd.DataFrame()
            now_df['uid'] = df['uid'].unique()
            now_df = now_df.merge(df.groupby('uid')['appId'].count().to_frame(), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['appId','use_date'].agg(['nunique']), how='left', on='uid')
            now_df = now_df.merge(df.groupby('uid')['duration','times'].agg(['mean','max','std']), how='left', on='uid')    

            now_df.columns = ['uid','usage_cnt','usage_appid_cnt','usage_date_cnt','duration_mean','duration_max','duration_std','times_mean','times_max','times_std']


            df = df.merge(app_info, how='left', on='appId')
            now_df = now_df.merge(df.groupby('uid')['category'].nunique().to_frame(), how='left', on='uid')
            #print(df.groupby(['uid'])['category'].value_counts().index[0])
            now_df['usage_most_used_category'] = df.groupby(['uid'])['category'].transform(f)
            resTable = pd.concat([resTable, now_df])
        except StopIteration:
            break
    
    resTable.to_csv("appuseProcessed.csv",index=0)
    
    print("Iterator is stopped")

In [ ]:
processUserAppActived()
processUserAppUsage()

In [4]:
#将user_basic_info.csv 和 user_behavior_info.csv中的字符值编码成可以训练的数值类型，合并
class2id = {}
id2class = {}
def mergeBasicTables(baseTable):
    resTable = baseTable.merge(user_basic_info, how='left', on='uid', suffixes=('_base0', '_ubaf'))
    resTable = resTable.merge(user_behavior_info, how='left', on='uid', suffixes=('_base1', '_ubef'))
    cat_columns = ['city','prodName','color','carrier','os','ct']
    for c in cat_columns:
        resTable[c] = resTable[c].apply(lambda x: x if type(x)==str else str(x))
        sort_temp = sorted(list(set(resTable[c])))  
        class2id[c+'2id'] = dict(zip(sort_temp, range(1, len(sort_temp)+1)))
        id2class['id2'+c] = dict(zip(range(1,len(sort_temp)+1), sort_temp))
        resTable[c] = resTable[c].apply(lambda x: class2id[c+'2id'][x])
        
    return resTable

In [5]:
#处理app使用相关数据
#对user_app_actived.csv简单统计
#将之前训练的appuseProcess.csv进行合并
def mergeAppData(baseTable):
    resTable = baseTable.merge(user_app_actived, how='left', on='uid')
    resTable['appId'] = resTable['appId'].apply(lambda x: len(list(x.split('#'))))
    appusedTable = pd.read_csv("appuseProcessed.csv")
    resTable = resTable.merge(appusedTable, how='left', on='uid')
    resTable[['category', 'usage_most_used_category']] = resTable[['category', 'usage_most_used_category']].fillna(41)
    resTable = resTable.fillna(0)
    #print(resTable[:5])
    appActivedTable = pd.read_csv("appActivedProcessed.csv")
    resTable = resTable.merge(appActivedTable, how='left', on='uid')
    return resTable

In [6]:
#合并用户基本特征以及app使用相关特征，作为训练集和测试集
df_train = mergeAppData(mergeBasicTables(age_train))
df_test = mergeAppData(mergeBasicTables(age_test))
print(df_train.shape)
print(df_test.shape)

(2010000, 74)
(502500, 73)


In [7]:
#训练模型

from sklearn.feature_selection import SelectFromModel, VarianceThreshold, SelectKBest, chi2, mutual_info_classif, f_classif
from sklearn.preprocessing import Imputer
from sklearn.ensemble import ExtraTreesClassifier

import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score
from sklearn.model_selection import StratifiedKFold

In [8]:
print("训练模型：")
param = {
        'learning_rate': 0.1,
        'lambda_l1': 0.1,
        'lambda_l2': 0.2,
        'max_depth': 20,
        'objective': 'multiclass',
        'num_class': 7,
        'num_leaves': 31,
        'min_data_in_leaf': 50,
        'max_bin': 230,
        'feature_fraction': 0.8,
        'metric': 'multi_error'
        }

X = df_train.drop(['age_group','uid'], axis=1)
y = df_train['age_group']
uid = df_test['uid']
test = df_test.drop('uid', axis=1)

xx_score = []
cv_pred = []
skf = StratifiedKFold(n_splits=5, random_state=1030, shuffle=True)
for index, (train_index, vali_index) in enumerate(skf.split(X, y)):
    print(index)
    x_train, y_train, x_vali, y_vali = np.array(X)[train_index], np.array(y)[train_index], np.array(X)[vali_index], np.array(y)[vali_index]
    train = lgb.Dataset(x_train, y_train)
    vali =lgb.Dataset(x_vali, y_vali)
    print("training start...")
    model = lgb.train(param, train, num_boost_round=1000, valid_sets=[vali], early_stopping_rounds=50)
    xx_pred = model.predict(x_vali,num_iteration=model.best_iteration)
    xx_pred = [np.argmax(x) for x in xx_pred]
    xx_score.append(f1_score(y_vali,xx_pred,average='weighted'))
    y_test = model.predict(test,num_iteration=model.best_iteration)
    y_test = [np.argmax(x) for x in y_test]
    if index == 0:
        cv_pred = np.array(y_test).reshape(-1, 1)
    else:
        cv_pred = np.hstack((cv_pred, np.array(y_test).reshape(-1, 1)))
    break;
        
submit = []
for line in cv_pred:
    submit.append(np.argmax(np.bincount(line)))
df = pd.DataFrame({'id':uid.as_matrix(),'label':submit})
df.to_csv('submission_new.csv',index=False)

训练模型：
0
training start...
[1]	valid_0's multi_error: 0.701493
Training until validation scores don't improve for 50 rounds.
[2]	valid_0's multi_error: 0.695313
[3]	valid_0's multi_error: 0.674405
[4]	valid_0's multi_error: 0.655784
[5]	valid_0's multi_error: 0.642545
[6]	valid_0's multi_error: 0.627075
[7]	valid_0's multi_error: 0.615373
[8]	valid_0's multi_error: 0.606458
[9]	valid_0's multi_error: 0.598331
[10]	valid_0's multi_error: 0.59155
[11]	valid_0's multi_error: 0.58593
[12]	valid_0's multi_error: 0.581786
[13]	valid_0's multi_error: 0.578129
[14]	valid_0's multi_error: 0.57447
[15]	valid_0's multi_error: 0.571391
[16]	valid_0's multi_error: 0.568891
[17]	valid_0's multi_error: 0.56598
[18]	valid_0's multi_error: 0.564177
[19]	valid_0's multi_error: 0.562331
[20]	valid_0's multi_error: 0.560386
[21]	valid_0's multi_error: 0.558425
[22]	valid_0's multi_error: 0.557219
[23]	valid_0's multi_error: 0.55541
[24]	valid_0's multi_error: 0.55393
[25]	valid_0's multi_error: 0.552403
[2

[218]	valid_0's multi_error: 0.497739
[219]	valid_0's multi_error: 0.497622
[220]	valid_0's multi_error: 0.497614
[221]	valid_0's multi_error: 0.497565
[222]	valid_0's multi_error: 0.497473
[223]	valid_0's multi_error: 0.497458
[224]	valid_0's multi_error: 0.497348
[225]	valid_0's multi_error: 0.497276
[226]	valid_0's multi_error: 0.497184
[227]	valid_0's multi_error: 0.497122
[228]	valid_0's multi_error: 0.49708
[229]	valid_0's multi_error: 0.496953
[230]	valid_0's multi_error: 0.496794
[231]	valid_0's multi_error: 0.496813
[232]	valid_0's multi_error: 0.496734
[233]	valid_0's multi_error: 0.496612
[234]	valid_0's multi_error: 0.496495
[235]	valid_0's multi_error: 0.496483
[236]	valid_0's multi_error: 0.496455
[237]	valid_0's multi_error: 0.496403
[238]	valid_0's multi_error: 0.496316
[239]	valid_0's multi_error: 0.496361
[240]	valid_0's multi_error: 0.496286
[241]	valid_0's multi_error: 0.496184
[242]	valid_0's multi_error: 0.496197
[243]	valid_0's multi_error: 0.496107
[244]	valid_0

[435]	valid_0's multi_error: 0.488547
[436]	valid_0's multi_error: 0.488498
[437]	valid_0's multi_error: 0.48846
[438]	valid_0's multi_error: 0.488488
[439]	valid_0's multi_error: 0.488396
[440]	valid_0's multi_error: 0.488465
[441]	valid_0's multi_error: 0.48844
[442]	valid_0's multi_error: 0.488433
[443]	valid_0's multi_error: 0.488373
[444]	valid_0's multi_error: 0.488311
[445]	valid_0's multi_error: 0.488251
[446]	valid_0's multi_error: 0.488284
[447]	valid_0's multi_error: 0.488318
[448]	valid_0's multi_error: 0.488308
[449]	valid_0's multi_error: 0.488274
[450]	valid_0's multi_error: 0.488259
[451]	valid_0's multi_error: 0.488246
[452]	valid_0's multi_error: 0.488246
[453]	valid_0's multi_error: 0.488169
[454]	valid_0's multi_error: 0.488147
[455]	valid_0's multi_error: 0.488047
[456]	valid_0's multi_error: 0.487985
[457]	valid_0's multi_error: 0.488015
[458]	valid_0's multi_error: 0.487988
[459]	valid_0's multi_error: 0.488137
[460]	valid_0's multi_error: 0.488172
[461]	valid_0'

[652]	valid_0's multi_error: 0.4859
[653]	valid_0's multi_error: 0.4859
[654]	valid_0's multi_error: 0.4859
[655]	valid_0's multi_error: 0.485851
[656]	valid_0's multi_error: 0.485851
[657]	valid_0's multi_error: 0.485853
[658]	valid_0's multi_error: 0.485871
[659]	valid_0's multi_error: 0.485891
[660]	valid_0's multi_error: 0.485943
[661]	valid_0's multi_error: 0.485891
[662]	valid_0's multi_error: 0.485873
[663]	valid_0's multi_error: 0.485886
[664]	valid_0's multi_error: 0.485886
[665]	valid_0's multi_error: 0.48591
[666]	valid_0's multi_error: 0.485891
[667]	valid_0's multi_error: 0.485881
[668]	valid_0's multi_error: 0.485888
[669]	valid_0's multi_error: 0.485856
[670]	valid_0's multi_error: 0.485803
[671]	valid_0's multi_error: 0.485803
[672]	valid_0's multi_error: 0.485794
[673]	valid_0's multi_error: 0.485781
[674]	valid_0's multi_error: 0.485704
[675]	valid_0's multi_error: 0.485687
[676]	valid_0's multi_error: 0.485734
[677]	valid_0's multi_error: 0.485766
[678]	valid_0's mul

[869]	valid_0's multi_error: 0.484547
[870]	valid_0's multi_error: 0.484515
[871]	valid_0's multi_error: 0.484465
[872]	valid_0's multi_error: 0.484463
[873]	valid_0's multi_error: 0.484485
[874]	valid_0's multi_error: 0.484425
[875]	valid_0's multi_error: 0.484435
[876]	valid_0's multi_error: 0.484403
[877]	valid_0's multi_error: 0.484391
[878]	valid_0's multi_error: 0.484356
[879]	valid_0's multi_error: 0.484393
[880]	valid_0's multi_error: 0.484373
[881]	valid_0's multi_error: 0.484388
[882]	valid_0's multi_error: 0.484405
[883]	valid_0's multi_error: 0.484398
[884]	valid_0's multi_error: 0.484415
[885]	valid_0's multi_error: 0.484386
[886]	valid_0's multi_error: 0.4844
[887]	valid_0's multi_error: 0.484363
[888]	valid_0's multi_error: 0.484361
[889]	valid_0's multi_error: 0.484351
[890]	valid_0's multi_error: 0.484331
[891]	valid_0's multi_error: 0.484321
[892]	valid_0's multi_error: 0.484348
[893]	valid_0's multi_error: 0.484353
[894]	valid_0's multi_error: 0.484391
[895]	valid_0'

c:\users\joze\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:45: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
